# Huggingface 

In [1]:
# from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
# embedding_provider = HuggingFaceEmbeddings(model_name='T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
# embedding_provider = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en', model_kwargs = {'device': 'cpu'})
# embedding_provider = HuggingFaceEmbeddings(model_name='deepset/gbert-large', model_kwargs = {'device': 'cpu'})
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, HuggingFaceInstructEmbeddings
# embedding_provider = HuggingFaceEmbeddings() # OpenAIEmbeddings()
# embedding_provider = HuggingFaceEmbeddings(model_name='sentence-transformers/distiluse-base-multilingual-cased-v1', model_kwargs = {'device': 'cpu'}) 
# embedding_provider = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

embedding_provider = HuggingFaceEmbeddings(model_name='T-Systems-onsite/cross-en-de-roberta-sentence-transformer', model_kwargs = {'device': 'cpu'}) 
# embedding_provider = HuggingFaceEmbeddings(model_name='sentence-transformers/distiluse-base-multilingual-cased-v1', model_kwargs = {'device': 'cpu'}) 

# model = SentenceTransformer('sentence-transformers/LaBSE')
# def embedding_provider(sentences):
#     embeddings = model.encode(sentences)
#     return embeddings

# model_name = "hkunlp/instructor-large"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# embedding_provider = HuggingFaceInstructEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

No sentence-transformers model found with name /home/hfwittmann/.cache/torch/sentence_transformers/T-Systems-onsite_cross-en-de-roberta-sentence-transformer. Creating a new one with MEAN pooling.


# Translate

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en")

In [ ]:
pipe("Wie geht es Dir?") 

[{'translation_text': 'How are you?'}]

## ... LLM

In [4]:
from langchain.llms import HuggingFacePipeline
llm_hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 128},
)

# Load

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://de.wikipedia.org/wiki/Angela_Merkel")
# loader = WebBaseLoader("https://en.wikipedia.org/wiki/Angela_Merkel")
data = loader.load()

# Split

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [5]:
print(all_splits[196])

page_content='Bundestagswahl 2021\nMerkel im Jahr 2021' metadata={'source': 'https://de.wikipedia.org/wiki/Angela_Merkel', 'title': 'Angela Merkel – Wikipedia', 'language': 'de'}


# Store

In [6]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding_provider)

# Retrieve

In [9]:
question = "Ist sie verheiratet, und mit wem?" # "What languages did Angela Merkel learn at school?" 
docs = vectorstore.similarity_search(question)
len(docs)
docs

[Document(page_content='Autor Reiner Kunze getroffen zu haben, den sie als ihren Lieblingsschriftsteller bezeichnet. 1977 heiratete sie den aus Cossengrün in Thüringen stammenden Physikstudenten Ulrich Merkel; die Ehe wurde 1982 geschieden.[12][13][14]', metadata={'source': 'https://de.wikipedia.org/wiki/Angela_Merkel', 'title': 'Angela Merkel – Wikipedia', 'language': 'de'}),
 Document(page_content='Familie\nWährend ihres Physikstudiums in Leipzig lernte Angela Kasner im Jahr 1974 bei einem Jugendaustausch mit Physikstudenten in Moskau und Leningrad ihren ersten Ehemann, den aus Cossengrün stammenden Physikstudenten Ulrich Merkel, kennen.[29] Am 3.\xa0September 1977 wurden die beiden in Templin kirchlich getraut. 1981 trennte sich das Paar und die kinderlose Ehe wurde 1982 in Ost-Berlin geschieden.[30] Ulrich Merkel hat aus seiner zweiten Ehe einen Sohn.[29]', metadata={'source': 'https://de.wikipedia.org/wiki/Angela_Merkel', 'title': 'Angela Merkel – Wikipedia', 'language': 'de'}),
 

In [10]:
embedding_vector = embedding_provider.embed_query(question)
docs = vectorstore.similarity_search_by_vector_with_relevance_scores(embedding_vector, k=4)
docs

[(Document(page_content='Autor Reiner Kunze getroffen zu haben, den sie als ihren Lieblingsschriftsteller bezeichnet. 1977 heiratete sie den aus Cossengrün in Thüringen stammenden Physikstudenten Ulrich Merkel; die Ehe wurde 1982 geschieden.[12][13][14]', metadata={'source': 'https://de.wikipedia.org/wiki/Angela_Merkel', 'title': 'Angela Merkel – Wikipedia', 'language': 'de'}),
  1.188572883605957),
 (Document(page_content='Familie\nWährend ihres Physikstudiums in Leipzig lernte Angela Kasner im Jahr 1974 bei einem Jugendaustausch mit Physikstudenten in Moskau und Leningrad ihren ersten Ehemann, den aus Cossengrün stammenden Physikstudenten Ulrich Merkel, kennen.[29] Am 3.\xa0September 1977 wurden die beiden in Templin kirchlich getraut. 1981 trennte sich das Paar und die kinderlose Ehe wurde 1982 in Ost-Berlin geschieden.[30] Ulrich Merkel hat aus seiner zweiten Ehe einen Sohn.[29]', metadata={'source': 'https://de.wikipedia.org/wiki/Angela_Merkel', 'title': 'Angela Merkel – Wikipedia

In [11]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [12]:
embedding_vector_question = embedding_provider.embed_query(question)

for ix, d in enumerate(all_splits):

    if ('Reiner Kunze getroffen zu haben' in d.page_content) or ('Physikstudiums in Leipzig' in d.page_content):
        embedding_vector_content = embedding_provider.embed_query(d.page_content)  
        similarity = cos_sim(embedding_vector_question, embedding_vector_content)
        
        print(ix, similarity, d.page_content)

25 0.2803985059413118 Autor Reiner Kunze getroffen zu haben, den sie als ihren Lieblingsschriftsteller bezeichnet. 1977 heiratete sie den aus Cossengrün in Thüringen stammenden Physikstudenten Ulrich Merkel; die Ehe wurde 1982 geschieden.[12][13][14]
36 0.2978068478652153 Familie
Während ihres Physikstudiums in Leipzig lernte Angela Kasner im Jahr 1974 bei einem Jugendaustausch mit Physikstudenten in Moskau und Leningrad ihren ersten Ehemann, den aus Cossengrün stammenden Physikstudenten Ulrich Merkel, kennen.[29] Am 3. September 1977 wurden die beiden in Templin kirchlich getraut. 1981 trennte sich das Paar und die kinderlose Ehe wurde 1982 in Ost-Berlin geschieden.[30] Ulrich Merkel hat aus seiner zweiten Ehe einen Sohn.[29]


# Generate

In [24]:
from langchain.chains import RetrievalQA

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [25]:
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever(), return_source_documents=True)
out = qa_chain({"query": question})
out

{'query': 'Ist sie verheiratet, und mit wem?',
 'result': 'Angela Merkel ist derzeit mit Joachim Sauer verheiratet. Sie haben am 30. Dezember 1998 geheiratet. Joachim Sauer ist ein Quantenchemiker.',
 'source_documents': [Document(page_content='Autor Reiner Kunze getroffen zu haben, den sie als ihren Lieblingsschriftsteller bezeichnet. 1977 heiratete sie den aus Cossengrün in Thüringen stammenden Physikstudenten Ulrich Merkel; die Ehe wurde 1982 geschieden.[12][13][14]', metadata={'source': 'https://de.wikipedia.org/wiki/Angela_Merkel', 'title': 'Angela Merkel – Wikipedia', 'language': 'de'}),
  Document(page_content='Familie\nWährend ihres Physikstudiums in Leipzig lernte Angela Kasner im Jahr 1974 bei einem Jugendaustausch mit Physikstudenten in Moskau und Leningrad ihren ersten Ehemann, den aus Cossengrün stammenden Physikstudenten Ulrich Merkel, kennen.[29] Am 3.\xa0September 1977 wurden die beiden in Templin kirchlich getraut. 1981 trennte sich das Paar und die kinderlose Ehe wurd

In [ ]:
# from llama_cpp import Llama
# llm = Llama(model_path="/home/hfwittmann/.cache/pip/wheels/aa/ed/39/87f2ad350dbbf13b600ac744899186b8647c5323c62e2bb348")
# output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)
